In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import cv2
import easyocr
import imutils
import matplotlib.pyplot as plt
%matplotlib inline

# capture live video strime
URL = "http://192.168.0.91:8080/video"
capture = cv2.VideoCapture(URL)
reader = easyocr.Reader(['en'])

while 1:

    # read video strime
    _, frame = capture.read()

    # Convert colored image into grayscale formate
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Filter and Edge Detection
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) # Edge detection

    # Find Contours and Apply Mask
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key = cv2.contourArea,reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour,10,True)
        if len(approx) == 4:
            location = approx
            break

    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)
    new_image = cv2.bitwise_and(frame, frame, mask=mask)

    (x, y) = np.where(mask == 255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    
#     plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

    # use easy ocr to read text
    result = reader.readtext(cropped_image)
    print(f"Number Plate : {result}")

    # show the video strime
    cv2.imshow('LiveStrimeScreen', frame)


    # to stop the strime
    if cv2.waitKey(1) == ord("q"):
        break

# close the resorces
capture.release()
cv2.destroyAllWindows()